In [1]:
import sys
sys.path.append("../agrilearn/submodules/commons/")
from agrilearn.commons.s3 import s3_utils
from agrilearn.crop_classification import evaluate_utils, yaml_utils, processing
import geopandas as gpd
import os
import pandas as pd

2025-04-19 00:48:04.942502: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-19 00:48:04.958466: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745023684.979178 3052432 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745023684.985172 3052432 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-19 00:48:05.005166: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## 1. Define Variables and Read Data

In [2]:
MAIN_PATH = "/agrilearn_app/datasets/SOYBEAN/2022_2023/"
GEOPACKAGE_RAW_PATH = os.path.join(MAIN_PATH, "geopackage/raw/SOYBEAN_66835.gpkg")

NEW_EOPATCH_PATH_PROCESSED = os.path.join(MAIN_PATH, "eopatch/processed/")
NEW_EOPATCH_PATH_INPUT_MODEL = os.path.join(MAIN_PATH, "eopatch/input_model/")

OLD_EOPATCH_PATH_PROCESSED = "/agrilearn_app/datasets/SOYBEAN/eopatch/processed/"
OLD_EOPATCH_PATH_INPUT_MODEL = "/agrilearn_app/datasets/SOYBEAN/eopatch/input_model/"

REPORT_PATH = os.path.join(MAIN_PATH, "reports")


LABEL_EOPATCH_LOCATION = "eopath_location" 

In [3]:
df = gpd.read_file(GEOPACKAGE_RAW_PATH)

In [4]:
df.shape

(66835, 73)

In [5]:
df.head()

,monitoring_class,period,fonte,state,area,micro,eopath_location,start_season,end_season,peak_start,...,obs_emergence,obs_senescence,obs_harvest,created_by,area_id,last_date_crop_rnn,crop_distance_score_sits,crop_confidence_maha_sits,set,geometry
0,SOYBEAN,2022/2023,Agrosatélite - Grãos Amazonia e Cerrado 2022/2023,DF,None,559.0,start_2022-10-01_end_2023-05-01_monitoring_cla...,2022-10-01,2023-05-01,2022-12-01,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-47.35296 -15.69628, -47.35297..."
1,SOYBEAN,2022/2023,Agrosatélite - Grãos Amazonia e Cerrado 2022/2023,DF,134693.4154669908,559.0,start_2022-10-01_end_2023-05-01_monitoring_cla...,2022-10-01,2023-05-01,2022-12-01,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-47.64978 -15.69533, -47.64978..."
2,SOYBEAN,2022/2023,Agrosatélite - Grãos Amazonia e Cerrado 2022/2023,DF,129433.0580092196,559.0,start_2022-10-01_end_2023-05-01_monitoring_cla...,2022-10-01,2023-05-01,2022-12-01,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-47.35802 -15.81954, -47.358 -..."
3,SOYBEAN,2022/2023,Agrosatélite - Grãos Amazonia e Cerrado 2022/2023,DF,388247.6241988475,559.0,start_2022-10-01_end_2023-05-01_monitoring_cla...,2022-10-01,2023-05-01,2022-12-01,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-47.52799 -15.94517, -47.52799..."
4,SOYBEAN,2022/2023,Agrosatélite - Grãos Amazonia e Cerrado 2022/2023,DF,651343.0730417219,559.0,start_2022-10-01_end_2023-05-01_monitoring_cla...,2022-10-01,2023-05-01,2022-12-01,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-47.53758 -15.91131, -47.53751..."


## 2. Processing Geopackage

### 2.1 Drop Columns NaN

In [6]:
shape_before = df.shape[1]
df.dropna(axis=1, how='all', inplace=True)
print(f"Removed Columns: {shape_before - df.shape[1]}, Percentage: {(shape_before - df.shape[1]) / shape_before * 100:.2f}%")

Removed Columns: 41, Percentage: 56.16%


### 2.2 Check NaN values

In [7]:
# Calcula a quantidade de registros nulos e a porcentagem de registros nulos
df_check_NaN = pd.concat([df.isna().sum(), df.isna().sum() / df.shape[0] * 100], axis=1)
df_check_NaN.columns = ['Null Count', 'NaN percentage']
df_check_NaN.sort_values('NaN percentage', ascending=False, inplace=True)
df_check_NaN.head()

,Null Count,NaN percentage
area,1,0.001496
monitoring_class,0,0.000000
fonte,0,0.000000
period,0,0.000000
state,0,0.000000


### 2.3 Transform all Geometry to Multipolygon

In [8]:
from shapely.geometry import Polygon, MultiPolygon

In [9]:
df['geometry'] = df['geometry'].apply(lambda geom: MultiPolygon([geom]) if isinstance(geom, Polygon) else geom)

### 2.4 Check Duplicates

In [10]:
df['duplicates_id'] = None
duplicados = df[df.duplicated(subset='eopath_location', keep=False)]
# Adicione um identificador para os duplicados
duplicados['duplicates_id'] = duplicados.groupby('eopath_location').ngroup()
# Combine os dados duplicados com o DataFrame original
df.update(duplicados)
df.sort_values('duplicates_id', inplace=True)

In [11]:
duplicados.shape

(0, 33)

In [12]:
df[df['duplicates_id']==0]

,monitoring_class,period,fonte,state,area,micro,eopath_location,start_season,end_season,peak_start,...,sampled_date,is_valid_metrics,sos_valid,pos_valid,eos_valid,los_valid,is_valid_POS,is_valid_LOS,geometry,duplicates_id


In [13]:
shape_before = df.shape[0]
df.drop_duplicates(subset=['geometry', 'period'], inplace=True)
print(f"Removed records: {shape_before - df.shape[0]}, Percentage: {(shape_before - df.shape[0]) / shape_before * 100:.2f}%")

Removed records: 0, Percentage: 0.00%


### 2.5 feature engineering

In [14]:
df['safra'] = df['period'].apply(lambda x: 'safrinha' if x.split('/')[0] == x.split('/')[1] else 'safra')

In [15]:
df['safra'].value_counts()

safra
safra    66835
Name: count, dtype: int64

## 3. Move eopatch to other

#### A) PROCESSED

In [16]:
df_results_processed = processing.check_load_eopatch_and_change_eopatch_dir(df,
                                        eopatch_path=OLD_EOPATCH_PATH_PROCESSED,
                                        new_eopatch_path=NEW_EOPATCH_PATH_PROCESSED,
                                        label_monitoring_class='monitoring_class',
                                        label_eopatch_path='eopath_location')

2025-04-19 00:48:53,052 - INFO - Iniciando a execução da função 'check_load_eopatch_and_change_eopatch_dir'


  0%|          | 0/66835 [00:00<?, ?it/s]

2025-04-19 03:13:53,885 - INFO - Tempo de execução da função 'check_load_eopatch_and_change_eopatch_dir': 8700.83 segundos


In [17]:
df_results_processed

,miss_geopackage,error_processed,rsync_output
0,start_2022-10-01_end_2023-05-01_monitoring_cla...,None,
1,start_2022-10-01_end_2023-05-01_monitoring_cla...,None,
2,start_2022-10-01_end_2023-05-01_monitoring_cla...,None,
3,start_2022-10-01_end_2023-05-01_monitoring_cla...,None,
4,start_2022-10-01_end_2023-05-01_monitoring_cla...,None,
...,...,...,...
66830,start_2022-09-01_end_2023-06-01_monitoring_cla...,None,
66831,start_2022-09-01_end_2023-06-01_monitoring_cla...,None,
66832,start_2022-09-01_end_2023-06-01_monitoring_cla...,None,
66833,start_2022-09-01_end_2023-06-01_monitoring_cla...,None,


#### B) INPUT_MODEL

In [18]:
df_results_input = processing.check_load_eopatch_and_change_eopatch_dir(df,
                                        eopatch_path=OLD_EOPATCH_PATH_INPUT_MODEL,
                                        new_eopatch_path=NEW_EOPATCH_PATH_INPUT_MODEL,
                                        label_monitoring_class='monitoring_class',
                                        label_eopatch_path='eopath_location')

2025-04-19 03:13:53,922 - INFO - Iniciando a execução da função 'check_load_eopatch_and_change_eopatch_dir'


  0%|          | 0/66835 [00:00<?, ?it/s]

2025-04-19 04:10:47,423 - INFO - Tempo de execução da função 'check_load_eopatch_and_change_eopatch_dir': 3413.50 segundos


In [19]:
df_results_input['miss_geopackage'].nunique()

66835

In [20]:
df_results_input['error_processed'].value_counts()

error_processed
EOPatch not found    1234
Name: count, dtype: int64

## 4. Check processed eopatch

In [21]:
string_name = processing.get_geopackage_name(df)
string_name

'SOYBEAN_66835'

#### A) PROCESSED

In [22]:
df_processed, df_overview_processed = processing.check_processed_eopatches_features_based_geopackage(df, 
                                                                                         eopatch_path=NEW_EOPATCH_PATH_PROCESSED,
                                                                                         label_eopatch_path=LABEL_EOPATCH_LOCATION)

2025-04-19 04:10:47,496 - INFO - Iniciando a execução da função 'check_processed_eopatches_features_based_geopackage'


  0%|          | 0/66835 [00:00<?, ?it/s]

2025-04-19 04:53:55,643 - INFO - Tempo de execução da função 'check_processed_eopatches_features_based_geopackage': 2588.15 segundos


In [23]:
df_overview_processed

,0
total_eopatches,66835.0
eopatch_processed_exists,66835.0
eopatches_not_exist,0.0
percentage_eopatches_not_exist,0.0
len_data_exist,66835.0
percentage_len_data_exist,100.0
len_bands_exist,66835.0
percentage_len_bands_exist,100.0
len_time_interval_exist,66835.0
percentage_len_time_interval_exist,100.0


In [24]:
os.makedirs(REPORT_PATH, exist_ok=True)
final_processed_report_name = f"{REPORT_PATH}/overview_{string_name}_processed.csv"
df_overview_processed.to_csv(final_processed_report_name)
final_processed_report_name

'/agrilearn_app/datasets/SOYBEAN/2022_2023/reports/overview_SOYBEAN_66835_processed.csv'

In [25]:
df_processed[df_processed['eopatch_processed_exists']==False]

,eopatch_location_id,eopatch_processed_exists,len_data,len_bands,len_time_interval,len_timestamp,error_processed


In [26]:
df_processed[df_processed['eopatch_processed_exists']==True]

,eopatch_location_id,eopatch_processed_exists,len_data,len_bands,len_time_interval,len_timestamp,error_processed
0,start_2022-10-01_end_2023-05-01_monitoring_cla...,True,4,10,2,16,None
1,start_2022-10-01_end_2023-05-01_monitoring_cla...,True,4,10,2,19,None
2,start_2022-10-01_end_2023-05-01_monitoring_cla...,True,4,10,2,17,None
3,start_2022-10-01_end_2023-05-01_monitoring_cla...,True,4,10,2,16,None
4,start_2022-10-01_end_2023-05-01_monitoring_cla...,True,4,10,2,16,None
...,...,...,...,...,...,...,...
66830,start_2022-09-01_end_2023-06-01_monitoring_cla...,True,4,10,2,30,None
66831,start_2022-09-01_end_2023-06-01_monitoring_cla...,True,4,10,2,23,None
66832,start_2022-09-01_end_2023-06-01_monitoring_cla...,True,4,10,2,24,None
66833,start_2022-09-01_end_2023-06-01_monitoring_cla...,True,4,10,2,22,None


#### B) INPUT_MODEL

In [27]:
df_input_model, df_overview_input = processing.check_input_model_eopatches_features_based_geopackage(df, 
                                                                                                     eopatch_path=NEW_EOPATCH_PATH_INPUT_MODEL,
                                                                                                     label_eopatch_path=LABEL_EOPATCH_LOCATION)

2025-04-19 04:53:55,877 - INFO - Iniciando a execução da função 'check_input_model_eopatches_features_based_geopackage'


  0%|          | 0/66835 [00:00<?, ?it/s]

2025-04-19 04:59:56,921 - INFO - Tempo de execução da função 'check_input_model_eopatches_features_based_geopackage': 361.04 segundos


In [28]:
df_overview_input

,0
total_eopatches,66835.00000
eopatch_input_model_exists,63807.00000
eopatches_not_exist,3028.00000
percentage_eopatches_not_exist,4.53056
classes_equivalent,0.00000
classes_not_equivalent,66835.00000
percentage_classes_equivalent,0.00000
percentage_classes_not_equivalent,100.00000


In [29]:
df_input_model[(df_input_model['label_unique'].notna()) & 
                (df_input_model['geopackage_class'] != df_input_model['label_unique'])]

,eopatch_location_id,geopackage_class,eopatch_input_model_exists,shape_X_data,label_unique,shape_labels,shape_timestamp,error_input_model
0,start_2022-10-01_end_2023-05-01_monitoring_cla...,SOYBEAN,True,"(2, 50, 3, 13)",algodao,"(2, 50)",7.0,None
1,start_2022-10-01_end_2023-05-01_monitoring_cla...,SOYBEAN,True,"(1, 50, 3, 13)",algodao,"(1, 50)",4.0,None
2,start_2022-10-01_end_2023-05-01_monitoring_cla...,SOYBEAN,True,"(3, 50, 3, 13)",algodao,"(3, 50)",7.0,None
3,start_2022-10-01_end_2023-05-01_monitoring_cla...,SOYBEAN,True,"(3, 50, 3, 13)",algodao,"(3, 50)",7.0,None
4,start_2022-10-01_end_2023-05-01_monitoring_cla...,SOYBEAN,True,"(1, 50, 3, 13)",algodao,"(1, 50)",5.0,None
...,...,...,...,...,...,...,...,...
66830,start_2022-09-01_end_2023-06-01_monitoring_cla...,SOYBEAN,True,"(6, 50, 3, 13)",algodao,"(6, 50)",10.0,None
66831,start_2022-09-01_end_2023-06-01_monitoring_cla...,SOYBEAN,True,"(6, 50, 3, 13)",algodao,"(6, 50)",10.0,None
66832,start_2022-09-01_end_2023-06-01_monitoring_cla...,SOYBEAN,True,"(7, 50, 3, 13)",algodao,"(7, 50)",11.0,None
66833,start_2022-09-01_end_2023-06-01_monitoring_cla...,SOYBEAN,True,"(7, 50, 3, 13)",algodao,"(7, 50)",11.0,None


In [30]:
os.makedirs(REPORT_PATH, exist_ok=True)
final_processed_report_name = f"{REPORT_PATH}/overview_{string_name}_input_model.csv"
df_overview_input.to_csv(final_processed_report_name)
final_processed_report_name

'/agrilearn_app/datasets/SOYBEAN/2022_2023/reports/overview_SOYBEAN_66835_input_model.csv'

## 5. Merge Data

In [31]:
df_overview_input

,0
total_eopatches,66835.00000
eopatch_input_model_exists,63807.00000
eopatches_not_exist,3028.00000
percentage_eopatches_not_exist,4.53056
classes_equivalent,0.00000
classes_not_equivalent,66835.00000
percentage_classes_equivalent,0.00000
percentage_classes_not_equivalent,100.00000


In [32]:
df_processed.shape

(66835, 7)

In [33]:
df_input_model.shape

(66835, 8)

In [34]:
# Usando pd.merge para juntar os DataFrames
df_merged = pd.merge(df, df_processed, left_on='eopath_location', right_on='eopatch_location_id', how='left')
df_merged.shape

(66835, 41)

In [35]:
df_full = pd.merge(df_merged, df_input_model, left_on='eopath_location', right_on='eopatch_location_id', how='left')
df_full.shape

(66835, 49)

## 6. Save data

In [36]:
result_string = processing.get_geopackage_name(df_full, label_monitoring_class='monitoring_class')
GEOPACKAGE_PROCESSED_PATH = os.path.join(GEOPACKAGE_RAW_PATH.split('/raw/')[0]+"/", "processed", result_string+ ".gpkg")
GEOPACKAGE_PROCESSED_PATH

'/agrilearn_app/datasets/SOYBEAN/2022_2023/geopackage/processed/SOYBEAN_66835.gpkg'

In [37]:
df_full.to_file(GEOPACKAGE_PROCESSED_PATH, 
                driver='GPKG', 
                engine='fiona')

2025-04-19 04:59:59,852 - WARNING - Skipping field because of invalid value: key='shape_X_data', value=(2, 50, 3, 13)
2025-04-19 04:59:59,853 - WARNING - Skipping field because of invalid value: key='shape_labels', value=(2, 50)
2025-04-19 04:59:59,856 - WARNING - Skipping field because of invalid value: key='shape_X_data', value=(1, 50, 3, 13)
2025-04-19 04:59:59,857 - WARNING - Skipping field because of invalid value: key='shape_labels', value=(1, 50)
2025-04-19 04:59:59,858 - WARNING - Skipping field because of invalid value: key='shape_X_data', value=(3, 50, 3, 13)
2025-04-19 04:59:59,858 - WARNING - Skipping field because of invalid value: key='shape_labels', value=(3, 50)
2025-04-19 04:59:59,873 - WARNING - Skipping field because of invalid value: key='shape_X_data', value=(5, 50, 3, 13)
2025-04-19 04:59:59,874 - WARNING - Skipping field because of invalid value: key='shape_labels', value=(5, 50)
2025-04-19 04:59:59,899 - WARNING - Skipping field because of invalid value: key='sh